<a href="https://colab.research.google.com/github/strangeworks/examples/blob/master/examples/optimization.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" /></a>

# Installation

In [ ]:
%pip install -q -U pip && pip install -q strangeworks-optimization

## Authentication

Your API token can be found in the [portal](https://portal.strangeworks.com).

### Google Colab

If running in Google Colab first set your API token as a secret environment variable in Colab. You can do this by clicking on the key icon on the left, then adding a key called `STRANGEWORKS_API_KEY` with your API token as the value.

Then, run the cell below to authenticate.

In [1]:
import strangeworks as sw
from google.colab import userdata

api_key = userdata.get('STRANGEWORKS_API_KEY')
sw.authenticate(api_key)

### Local

If running locally, you can save your API token in a dotenv file. Create a file called `.env` in the same directory as this notebook and add the following line to the file:

```
STRANGEWORKS_API_KEY=your_api_token
```

Then, run the cell below to authenticate.

In [ ]:
# %pip install -q python-dotenv
# import strangeworks as sw
# import os
# from dotenv import load_dotenv

# load_dotenv()
# api_key = os.getenv("STRANGEWORKS_API_KEY")
# sw.authenticate(api_key)

# StrangeworksOptimizer

We implement our own Optimization class called `StrangeworksOptimizer`.
`StrangeworksOptimizer` provides the full functionality of the Strangeworks Optimization Service.

In [2]:
from strangeworks_optimization import StrangeworksOptimizer
so = StrangeworksOptimizer()

## Models

The service accepts the following models:

Standard:
- `BinaryQuadraticModel` (all but Aquila)

MPS:
- `MPSFile` (Gurobi, Quantagonia)

Solver Specific:
- `ConstrainedQuadraticModel` (D-Wave)
- `DiscreteQuadraticModel`(D-Wave)
- `QuboDict` (NEC)
- `HitachiModelList` (Hitachi)
- `FujitsuModelList`(Fujitsu)
- `JiJProblem` (JiJ)
- `AquilaNDArray` (Aquila)

[`dimod`](https://docs.ocean.dwavesys.com/en/stable/docs_dimod/) provides a shared API for samplers and we default to using `BinaryQuadraticModel` to store and represent QUBO problems.

Here is a simple QUBO problem defined by linear and quadratic terms. The linear terms are a dictionary where the keys are the variables and the values are the linear coefficients. The quadratic terms are a dictionary where the keys are the pairs of variables and the values are the quadratic coefficients. The BinaryQuadraticModel class is used to create the QUBO problem.

In [3]:
from dimod import BinaryQuadraticModel

linear = {1: -2, 2: -2, 3: -3, 4: -3, 5: -2}
quadratic = {(1, 2): 2, (1, 3): 2, (2, 4): 2, (3, 4): 2, (3, 5): 2, (4, 5): 2}
model = BinaryQuadraticModel(linear, quadratic, "BINARY")

## Solvers

The solver name usually follows the convention of `provider-id.backend-name`.

You can determine which solver `backends` are available to you through our platform. Check it out.

In [4]:
backends = optimizer.backends()

from pprint import pprint
pprint(backends)

[Backend(id='MzFiNTA4NWQtNDBkMi00YjEzLTg5ZjYtZmNmMjRlMjQxMDYz',
         data={},
         data_schema='',
         name='dwave.hybrid_binary_quadratic_model_version2p',
         remote_backend_id='hybrid_binary_quadratic_model_version2p',
         remote_status='ONLINE',
         slug='c3zfoarsv'),
 Backend(id='MzcwMWYxOGQtODEzOS00MWM2LWIwOGUtMThhMTI4NzhjZmJm',
         data={},
         data_schema='',
         name='dwave.Advantage2_prototype2.2',
         remote_backend_id='Advantage2_prototype2.2',
         remote_status=None,
         slug='mhm44b6t3'),
 Backend(id='NDYxYzJmNjAtNTk2NC00N2M5LTliNDgtMDg2YzFkMjgxMTEy',
         data={},
         data_schema='',
         name='dwave.Advantage2_prototype1.1',
         remote_backend_id='Advantage2_prototype1.1',
         remote_status='ONLINE',
         slug='3wj1kt2as'),
 Backend(id='NWJhY2Q0NjctOTJjNS00ZTBmLWE5MzUtODhlNDJjODZhZDI1',
         data={},
         data_schema='',
         name='dwave.Advantage_system6.3',
         remote

## Options

Lets use the `hitachi.cmos_annealer` backend, we can set a number of [values](https://annealing-cloud.com/en/web-api/reference/v2.html) to submit with the solver.

- `solver_type`: An integer value representing the type of solver.
- `num_executions`: An integer value representing the number of executions. It defaults to 1 if not specified.
- `temperature_num_steps`: An integer value representing the number of steps in the temperature. It defaults to 10 if not specified.
- `temperature_step_length`: An integer value representing the length of each step in the temperature. It defaults to 100 if not specified.
- `temperature_initial`: A float value representing the initial temperature. It defaults to 10.0 if not specified.
- `temperature_target`: A float value representing the target temperature. It defaults to 0.01 if not specified.
- `energies`: A boolean value indicating whether to include energies in the output. It defaults to True if not specified.
spins: A boolean value indicating whether to include spins in the output. It defaults to True if not specified.
-  `execution_time`: A boolean value indicating whether to include execution time in the output. It defaults to True if not specified.
- `num_outputs`: An integer value representing the number of outputs. It defaults to 0 if not specified.
- averaged_spins: A boolean value indicating whether to include averaged spins in the output. It defaults to False if not specified.
- `averaged_energy`: A boolean value indicating whether to include averaged energy in the output. It defaults to False if not specified.

[![Hitachi CMOS annealing machine](https://annealing-cloud.com/images/memory-cell-array-en.png)](https://annealing-cloud.com/en/about/cmos-annealing-machine.html)

In [5]:
solver = "hitachi.cmos_annealer"

from strangeworks_optimization_models.parameter_models import HitachiParameterModel

options = HitachiParameterModel(solver_type=5)

## Running a job

In [ ]:
tags = ["foo", "bar"]
so = StrangeworksOptimizer(model=model, solver=solver, options=options, tags=tags)
sw_job = so.run()
print(f"Job slug: {sw_job.slug}")

## Status

In [7]:
print(f"Job status: {so.status(sw_job.slug)}")

Job status: CREATED


## Results

In [8]:
results = so.results(sw_job.slug)
print(f"Best solution:\n{results.solution.first}")

Best solution:
Sample(sample={1: 0, 2: 1, 3: 1, 4: 0, 5: 1}, energy=-5.0, num_occurrences=1)
